You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    # What is the role it's playing
    role="Content Planner",
    
    # What we want to do using the prompt
    goal="Plan engaging and factually accurate content on {topic}",
    
    # prompt engineering kind of conveying it as a blogger or a writer, etc
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    
    # to see how it runs, should be disable in real-time
    # it's only for use to understand log stream output
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    
    # goal
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    
    # prompt
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    
    # What we want to do
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    
    # prompt to further edit the content generate from above two
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:


plan = Task(
    
    # What we expect the task to do or agent to do
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    
    # Output of task
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    
    # assign a agent
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    
    # What we expect the task to do or agent to do
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    
    # Output of task
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    
    # Assignt an agent
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    
    # What we expect the task to do or agent to do
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    
    # Output of task
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    
    # Assignt an agent
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:

# Crew operates sequentially the order in which you assign tasks is important
crew = Crew(
    
    # List of Agents
    agents=[planner, writer, editor],
    
    # List of Tasks
    # the order in which you assign tasks is important as it operates sequentially
    tasks=[plan, write, edit],
    
    # verbose mode which would help us see who it's running explore logs
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Outline:
I. Introduction
- Brief overview of artificial intelligence
- Importance of staying updated on the latest trends and news in AI

II. Latest Trends in Artificial Intelligence
- Evolution of deep learning and neural networks
- Advancements in natural language processing
- Growth of AI in healthcare, finance, and autonomous vehicles

III. Key Players in the AI Industry
- Google, Amazon

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial intelligence (AI) has undoubtedly become a focal point in today's technological world, reshaping industries and paving the way for future innovations. Staying abreast of the latest trends and developments in the AI realm is essential for both individuals and businesses looking to capitalize on the potential that AI holds. The rapid evolution of AI has the power to revolutionize how we live and work, underscoring the importance of understanding the key players driving this technological advancement.

In the realm of artificial intelligence, deep learning and neural networks have emerged as significant trends, empowering machines to learn from data and execute complex tasks akin to human capabilities. Furthermore, the progress in natural language processing has been remarkable, enabling AI systems to comprehend and generate human language with 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Artificial intelligence (AI) has undoubtedly become a focal point in today's technological world, reshaping industries and paving the way for future innovations. Staying abreast of the latest trends and developments in the AI realm is essential for both individuals and businesses looking to capitalize on the potential that AI holds. The rapid evolution of AI has the power to revolutionize how we live and work, underscoring the importance of understanding the key players driving this technological advancement.

In the realm of artificial intelligence, deep learning and neural networks have emerged as significant trends, empowering machines to learn from data and execute complex tasks akin to human capabilities. Furthermore, the progress in natural language processing has been remarkable, enabling AI systems to comprehend and generate human language with remarkable proficiency. The application of AI in critical sectors such as healthcare, finance, and autonomous vehicles has seen substantial growth, showcasing the diverse and far-reaching possibilities that AI presents across various industries.

The landscape of the AI industry is largely shaped by tech giants like Google, Amazon, and Microsoft, who are at the forefront of driving AI innovation with cutting-edge technologies and solutions that redefine the boundaries of AI capabilities. In addition to these industry leaders, startups with inventive ideas and novel approaches are disrupting the AI space and playing a pivotal role in shaping the future of the industry. Research institutions dedicated to AI research and development also play a crucial part in propelling innovation and exploring the full potential of artificial intelligence.

Recent breakthroughs in AI research, particularly advancements in machine learning algorithms and ethical considerations in AI development, have dominated headlines within the industry. Discussions surrounding ethical issues such as bias, fairness, and the responsible use of AI technologies have sparked critical conversations. The impact of AI on employment and society has also been a key focal point, with ongoing dialogues about how AI will redefine the workforce and the broader implications for society as a whole.

As the field of artificial intelligence continues to progress at a rapid pace, it is imperative for tech enthusiasts, researchers, professionals, and businesses to remain informed about the latest trends and advancements. By engaging with reputable sources, industry leaders, and credible resources, individuals can gather valuable insights and make well-informed decisions regarding the future of AI technology. The potential of artificial intelligence is vast, and through active participation in discussions and staying informed, we can collectively shape a future where AI enhances society on the whole.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Meditation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Meditation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Power of Meditation: A Guide to Finding Inner Peace

Outline:

I. Introduction
- Brief explanation of what meditation is
- Importance of incorporating meditation into daily routine

II. Latest Trends in Meditation
- Rise of meditation apps and online platforms
- Integration of meditation in workplace wellness programs
- Research studies on the benefits of meditation

III. Key Players in the Meditation Industry
- Insight Timer
- Headspace
- Calm
- Deepak Chopra

IV. Noteworthy News on Meditation

I now can give a great answer

Final Answer:

# The Power of Meditation: A Guide to Finding Inner Peace

Meditation, a practice steeped in centuries of tradition and spirituality, offers individuals a path to inner peace and self-awareness. By honing the mind and fostering mindfulness, meditation serves as a sanctuary of calm in a world characterized by chaos and incessant activity. Integrating meditation into your daily routine can yield a myriad of benefits for your mental, emotional, and physical well-being, providing a refuge for moments of tranquility and clarity amidst the hustle and bustle of modern life.

In the realm of meditation, recent years have witnessed a surge in popularity, buoyed by the advent of meditation apps and online platforms that have democratised access to this ancient practice. Moreover, corporations are increasingly incorporating meditation into their wellness initiatives, recognising its capacity to bolster employee productivity and satisfaction. Research 

In [14]:
Markdown(result)

# The Power of Meditation: A Guide to Finding Inner Peace

Meditation, a practice steeped in centuries of tradition and spirituality, offers individuals a path to inner peace and self-awareness. By honing the mind and fostering mindfulness, meditation serves as a sanctuary of calm in a world characterized by chaos and incessant activity. Integrating meditation into your daily routine can yield a myriad of benefits for your mental, emotional, and physical well-being, providing a refuge for moments of tranquility and clarity amidst the hustle and bustle of modern life.

In the realm of meditation, recent years have witnessed a surge in popularity, buoyed by the advent of meditation apps and online platforms that have democratised access to this ancient practice. Moreover, corporations are increasingly incorporating meditation into their wellness initiatives, recognising its capacity to bolster employee productivity and satisfaction. Research substantiates the efficacy of meditation in stress reduction, focus enhancement, and overall well-being, underscoring its transformative potential in fostering a balanced and harmonious existence.

In the landscape of the meditation industry, prominent players such as Insight Timer, Headspace, Calm, and Deepak Chopra have emerged as purveyors of guided meditation sessions, mindfulness practices, and tools designed to support individuals on their meditative odyssey. Whether you are a neophyte or a seasoned practitioner, these resources are invaluable in deepening your meditation practice and harnessing its profound therapeutic effects.

## Latest Trends in Meditation

In recent years, there has been a significant rise in the popularity of meditation, with the emergence of meditation apps and online platforms making it more accessible to a wider audience. Companies are also integrating meditation into their workplace wellness programs, recognizing the positive impact it can have on employee productivity and satisfaction. Research studies continue to demonstrate the benefits of meditation, showing its effectiveness in reducing stress, improving focus, and enhancing overall well-being.

## Key Players in the Meditation Industry

Some of the key players in the meditation industry include Insight Timer, Headspace, Calm, and Deepak Chopra. These platforms offer a variety of guided meditation sessions, mindfulness practices, and tools to support individuals in their meditation journey. Whether you're a beginner or a seasoned meditator, these resources can help you deepen your practice and experience the transformative power of meditation.

## Noteworthy News on Meditation

Recent studies have highlighted the impact of meditation on mental health, showing its effectiveness in reducing symptoms of anxiety and depression. Celebrities have also been vocal about their advocacy for meditation practices, sharing how it has positively impacted their lives. Events and workshops focused on meditation and mindfulness continue to attract individuals seeking to cultivate a sense of inner peace and well-being.

## Target Audience Analysis

Meditation appeals to a wide range of individuals, including young professionals looking to reduce stress and improve focus, stay-at-home parents seeking moments of relaxation, individuals dealing with anxiety or depression, and health and wellness enthusiasts interested in holistic practices. Whether you're new to meditation or have been practicing for years, there are resources and tools available to support you on your journey towards greater well-being.

## SEO Keywords

When it comes to meditation, keywords such as "meditation benefits," "mindfulness practices," "guided meditation techniques," "meditation for beginners," and "meditation apps and tools" can help individuals find valuable resources and information to enhance their practice. By incorporating these keywords into your search queries, you can discover a wealth of resources to support your meditation journey.

## Call to Action

As you explore the world of meditation, I encourage you to start incorporating meditation into your daily routine. Whether you prefer guided meditation sessions or mindfulness practices, there are resources available to support you on your journey towards inner peace and well-being. I invite you to share your meditation experiences in the comments section below and connect with others who are also seeking to find stillness and clarity in their lives.

## Resources

For those looking to dive deeper into the benefits of meditation, scientific studies, meditation apps, online platforms, and books and articles by industry experts can provide valuable insights and guidance. By exploring these resources, you can enhance your meditation practice and experience the transformative power of mindfulness in your daily life.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).